# Notebook with GTFS methods

Goals: 

* Make a way to calculate the scheduled number of current active trips given a date, time, and route. 
    - Take datetime and find what services are active on that date 
    - Find what trips run on those services + route 
    - Find which of those trips are "in progress" per stop_times
* ~Output most common shape by route~

In [1]:
# imports 

import boto3
import os
import pandas as pd
import zipfile
import requests
import pendulum
from io import BytesIO
import shapely
import geopandas

In [2]:
# if "private", will assume you have write permissions and allow you to write; else will not attempt to write files
BUCKET_TYPE = "private"

In [3]:
# local 
# CTA_GTFS = zipfile.ZipFile('cta_gtfs_20220509.zip')
# s3
# follow https://pythonguides.com/download-zip-file-from-url-using-python/
# CTA_GTFS = zipfile.ZipFile(BytesIO(requests.get('https://chn-ghost-buses-public.s3.us-east-2.amazonaws.com/cta_static_gtfs/cta_gtfs_20220509.zip').content))
# cta website

# VERSION_ID = '20220718'

RTD_GTFS = zipfile.ZipFile('../../utils/utils/gtfs/google_transit_20231504.zip') # 

In [17]:
class GTFSFeed:
   """ Static GTFS management """
   def __init__(self, gtfs_zipfile):
        self.gtfs_zipfile = gtfs_zipfile
        self.feed_start = None
        self.feed_end = None
        try: 
            with self.gtfs_zipfile.open('stops.txt') as file:
                    self.stops = pd.read_csv(file, dtype = 'object')
                    print("stops.txt loaded")
            with self.gtfs_zipfile.open('stop_times.txt') as file:
                    self.stop_times = pd.read_csv(file, dtype = 'object')
                    print("stop_times.txt loaded")
            with self.gtfs_zipfile.open('routes.txt') as file:
                    self.routes = pd.read_csv(file, dtype = 'object')
                    print("routes.txt loaded")
            with self.gtfs_zipfile.open('trips.txt') as file:
                    self.trips = pd.read_csv(file, dtype = 'object')
                    print("trips.txt loaded")
        except KeyError as e:
            print("GTFS is missing required file")
            print(e)
        if 'calendar.txt' in self.gtfs_zipfile.namelist():
                with self.gtfs_zipfile.open('calendar.txt') as file:
                        self.calendar = pd.read_csv(file, dtype = 'object')
                        print("calendar.txt loaded")
        else:
            print("no calendar.txt found")
        if 'calendar_dates.txt' in self.gtfs_zipfile.namelist():
                with self.gtfs_zipfile.open('calendar_dates.txt') as file:
                        self.calendar_dates = pd.read_csv(file, dtype = 'object')
                        print("calendar_dates.txt loaded")
        else:
            print("no calendar_dates.txt found")
        if 'shapes.txt' in self.gtfs_zipfile.namelist():
                with self.gtfs_zipfile.open('shapes.txt') as file:
                        self.shapes = pd.read_csv(file, dtype = 'object')
                        print("shapes.txt loaded")
        else:
            print("no shapes.txt found")
        if 'feed_info.txt' in self.gtfs_zipfile.namelist():
                with self.gtfs_zipfile.open('feed_info.txt') as file:
                        self.feed_info = pd.read_csv(file, dtype = 'object')
                        print("feed_info.txt loaded")
                self.feed_start = pd.to_datetime(self.feed_info['feed_start_date'][0])
                self.feed_end = pd.to_datetime(self.feed_info['feed_end_date'][0])
        else:
            print("no feed_info.txt found")
            

In [18]:
data = GTFSFeed(RTD_GTFS)

stops.txt loaded
stop_times.txt loaded
routes.txt loaded
trips.txt loaded
calendar.txt loaded
calendar_dates.txt loaded
shapes.txt loaded
feed_info.txt loaded


In [20]:
data.feed_end

Timestamp('2023-08-19 00:00:00')

In [27]:
# TODO: Convert calen
data.calendar
# data.trips
# data.stop_times

,service_id,start_date,end_date,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,SA_merged_114569136,20230528,20230819,0,0,0,0,0,1,0
1,SA_3340,20230403,20230501,0,0,0,0,0,1,0
2,MT_3340,20230403,20230501,1,1,1,1,0,0,0
3,WK_3340,20230403,20230501,1,1,1,1,1,0,0
4,FR_merged_114569130,20230528,20230819,0,0,0,0,1,0,0
5,MT_merged_114569127,20230108,20230527,1,1,1,1,0,0,0
6,FR_merged_114569123,20230108,20230527,0,0,0,0,1,0,0
7,FR_3340,20230403,20230501,0,0,0,0,1,0,0
8,WK_merged_114569132,20230528,20230819,1,1,1,1,1,0,0
9,WK_merged_114569125,20230108,20230527,1,1,1,1,1,0,0


In [34]:
calendar_date_range = pd.DataFrame(pd.date_range(data.feed_start, data.feed_end), columns = ['raw_date'])
    
#     # cross join calendar index with actual calendar to get all combos of possible dates & services 
calendar_cross = calendar_date_range.merge(data.calendar, how = "cross")

#     # extract day of week from date index date
calendar_cross['dayofweek'] = calendar_cross['raw_date'].dt.dayofweek
calendar_cross

    # take wide calendar data (one col per day of week) and make it long (one row per day of week)
actual_service = calendar_cross.melt(id_vars = ['raw_date', 'start_date', 'end_date', 'service_id', 'dayofweek'], var_name = 'cal_dayofweek', value_name = 'cal_val')

# #     # map the calendar input strings to day of week integers to align w pandas dayofweek output
actual_service['cal_daynum'] = actual_service['cal_dayofweek'].map({
    'monday': 0,
    'tuesday': 1,
    'wednesday': 2,
    'thursday': 3,
    'friday': 4,
    'saturday': 5,
    'sunday': 6
})

In [6]:
data.calendar

,service_id,start_date,end_date,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,SA_merged_114569136,20230528,20230819,0,0,0,0,0,1,0
1,SA_3340,20230403,20230501,0,0,0,0,0,1,0
2,MT_3340,20230403,20230501,1,1,1,1,0,0,0
3,WK_3340,20230403,20230501,1,1,1,1,1,0,0
4,FR_merged_114569130,20230528,20230819,0,0,0,0,1,0,0
5,MT_merged_114569127,20230108,20230527,1,1,1,1,0,0,0
6,FR_merged_114569123,20230108,20230527,0,0,0,0,1,0,0
7,FR_3340,20230403,20230501,0,0,0,0,1,0,0
8,WK_merged_114569132,20230528,20230819,1,1,1,1,1,0,0
9,WK_merged_114569125,20230108,20230527,1,1,1,1,1,0,0


In [7]:
data.calendar_dates

,service_id,date,exception_type
0,WK_merged_114569132,20230704,2
1,WK_merged_114569132,20230529,2
2,P_Fri_merged_114569135,20230704,2
3,P_Fri_merged_114569135,20230529,2
4,DPSWK_merged_114569131,20230704,2
5,DPSWK_merged_114569131,20230529,2
6,SU_merged_114569133,20230704,1
7,SU_merged_114569133,20230529,1
8,MT_merged_114569134,20230704,2
9,MT_merged_114569134,20230529,2


## Basic data transformations

Ex. creating actual timestamps

In [8]:
# def make_timestamp(s, date):
#     parts = s.split(':')
#     assert len(parts)==3
#     if int(parts[0]) > 23:
#         num_parts = [int(parts[0]) - 24, int(parts[1]), int(parts[2])]
#     else:
#         num_parts = [int(parts[0]), int(parts[1]), int(parts[2])]
#     return pendulum.datetime(year = date.year, month = date.month, day = date.day, hour = num_parts[0], minute = num_parts[1], second = num_parts[2])
        

In [38]:
def get_hour(s):
    parts = s.split(':')
    assert len(parts)==3
    hour = int(parts[0])
    if hour >= 24:
        hour -= 24
    return hour

In [40]:
def format_dates_hours(data):
    # convert string dates to actual datetimes in calendar.txt and calendar_dates.txt
    data.calendar['start_date_dt'] = data.calendar['start_date'].apply(lambda x: pendulum.from_format(x, 'YYYYMMDD'))
    data.calendar['end_date_dt'] = data.calendar['end_date'].apply(lambda x: pendulum.from_format(x, 'YYYYMMDD'))
    data.calendar_dates['date_dt'] = data.calendar_dates['date'].apply(lambda x: pendulum.from_format(x, 'YYYYMMDD'))
    
    # extract hour from stop_times timestamps 
    data.stop_times['arrival_hour'] = data.stop_times.arrival_time.apply(lambda x: get_hour(x))
    data.stop_times['departure_hour'] = data.stop_times.departure_time.apply(lambda x: get_hour(x))
    
    return data

In [41]:
data = format_dates_hours(data)

In [42]:
# check that there are no dwell periods that cross hour boundary
# 476 rows - arrive at 59, leave a minute or two later. 476 instances.
data.stop_times[data.stop_times.arrival_hour != data.stop_times.departure_hour]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_hour,departure_hour
651,114458892,19:59:00,20:02:00,34534,26,NaN,NaN,NaN,NaN,NaN,19,20
2317,114357406,06:59:00,07:00:00,13703,22,NaN,NaN,NaN,NaN,NaN,6,7
2389,114357407,16:59:00,17:00:00,13703,22,NaN,NaN,NaN,NaN,NaN,16,17
6379,114443077,14:59:00,15:00:00,34539,42,NaN,NaN,NaN,NaN,NaN,14,15
6439,114443076,13:59:00,14:00:00,34539,42,NaN,NaN,NaN,NaN,NaN,13,14
...,...,...,...,...,...,...,...,...,...,...,...,...
1439032,114454314,20:59:00,21:00:00,13703,22,NaN,NaN,NaN,NaN,NaN,20,21
1439334,114454311,07:59:00,08:00:00,13703,22,NaN,NaN,NaN,NaN,NaN,7,8
1441727,114360586,20:59:00,21:00:00,35033,1,NaN,NaN,1,NaN,NaN,20,21
1441829,114568212,22:59:45,23:03:15,23061,7,NaN,NaN,NaN,NaN,NaN,22,23


In [43]:
def make_trip_summary(data):
    # construct a datetime index that has every day between calendar start and end 
    calendar_date_range = pd.DataFrame(pd.date_range(min(data.calendar.start_date_dt), max(data.calendar.end_date_dt)), columns = ['raw_date'])
    
    # cross join calendar index with actual calendar to get all combos of possible dates & services 
    calendar_cross = calendar_date_range.merge(data.calendar, how = "cross")
    
    # extract day of week from date index date
    calendar_cross['dayofweek'] = calendar_cross['raw_date'].dt.dayofweek
    
    # take wide calendar data (one col per day of week) and make it long (one row per day of week)
    actual_service = calendar_cross.melt(id_vars = ['raw_date', 'start_date_dt', 'end_date_dt', 'start_date', 'end_date', 'service_id', 'dayofweek'], var_name = 'cal_dayofweek', value_name = 'cal_val')
    
    # map the calendar input strings to day of week integers to align w pandas dayofweek output
    actual_service['cal_daynum'] = actual_service['cal_dayofweek'].map({
        'monday': 0,
        'tuesday': 1,
        'wednesday': 2,
        'thursday': 3,
        'friday': 4,
        'saturday': 5,
        'sunday': 6
    })
    
    # now check for rows that "work"
    # i.e., the day of week matches between datetime index & calendar input
    # and the datetime index is between the calendar row's start and end dates
    actual_service = actual_service[(actual_service.dayofweek == actual_service.cal_daynum) & 
                                   (actual_service.start_date_dt <= actual_service.raw_date) &
                                   (actual_service.end_date_dt >= actual_service.raw_date)]
    
    # now merge in calendar dates to the datetime index to get overrides
    actual_service = actual_service.merge(data.calendar_dates, how = 'outer', left_on = ['raw_date', 'service_id'], right_on = ['date_dt', 'service_id'])
    
    # now add a service happened flag for dates where the schedule indicates that this service occurred
    # i.e.: calendar has a service indicator of 1 and there's no exception type from calendar_dates
    # OR calendar_dates has exception type of 1
    # otherwise no service 
    # https://stackoverflow.com/questions/21415661/logical-operators-for-boolean-indexing-in-pandas
    actual_service['service_happened'] = ((actual_service['cal_val'] == '1') & 
                                          actual_service['exception_type'].isnull()) | (actual_service['exception_type'] == '1')

    
    # now fill in rows where calendar_dates had a date outside the bounds of the datetime index, so raw_date is always populated
    actual_service['raw_date'] = actual_service['raw_date'].fillna(actual_service['date_dt'])
    
    # filter to only rows where service occurred
    service_happened = actual_service[actual_service.service_happened]
    
    # join trips to only service that occurred
    trips_happened = data.trips.merge(service_happened, how = 'left', on = 'service_id')
    
    # get only the trip / hour combos that actually occurred
    trip_stop_hours = data.stop_times[['trip_id', 'arrival_hour']].drop_duplicates()
    
    # now join
    # result has one row per date + row from trips.txt (incl. route) + hour
    trip_summary = trips_happened.merge(trip_stop_hours, how = "left", on = "trip_id")
    
    return trip_summary
    
    

In [44]:
trip_summary = make_trip_summary(data)

In [45]:
print(trip_summary.dtypes)
print(trip_summary.shape)
trip_summary.head()

block_id                                     object
route_id                                     object
direction_id                                 object
trip_headsign                                object
shape_id                                     object
service_id                                   object
trip_id                                      object
raw_date            datetime64[ns, Timezone('UTC')]
start_date_dt       datetime64[ns, Timezone('UTC')]
end_date_dt         datetime64[ns, Timezone('UTC')]
start_date                                   object
end_date                                     object
dayofweek                                     int64
cal_dayofweek                                object
cal_val                                      object
cal_daynum                                    int64
date                                         object
exception_type                               object
date_dt             datetime64[ns, Timezone('UTC')]
service_happ

,block_id,route_id,direction_id,trip_headsign,shape_id,service_id,trip_id,raw_date,start_date_dt,end_date_dt,...,end_date,dayofweek,cal_dayofweek,cal_val,cal_daynum,date,exception_type,date_dt,service_happened,arrival_hour
0,b_76 4,76,0,US36 & Bfld,1241914,SU_merged_114569133,114458891,2023-05-29 00:00:00+00:00,2023-05-28 00:00:00+00:00,2023-08-19 00:00:00+00:00,...,20230819,0,monday,0,0,20230529,1,2023-05-29 00:00:00+00:00,True,8
1,b_76 4,76,0,US36 & Bfld,1241914,SU_merged_114569133,114458891,2023-05-29 00:00:00+00:00,2023-05-28 00:00:00+00:00,2023-08-19 00:00:00+00:00,...,20230819,0,monday,0,0,20230529,1,2023-05-29 00:00:00+00:00,True,9
2,b_76 4,76,0,US36 & Bfld,1241914,SU_merged_114569133,114458891,2023-07-04 00:00:00+00:00,2023-05-28 00:00:00+00:00,2023-08-19 00:00:00+00:00,...,20230819,1,tuesday,0,1,20230704,1,2023-07-04 00:00:00+00:00,True,8
3,b_76 4,76,0,US36 & Bfld,1241914,SU_merged_114569133,114458891,2023-07-04 00:00:00+00:00,2023-05-28 00:00:00+00:00,2023-08-19 00:00:00+00:00,...,20230819,1,tuesday,0,1,20230704,1,2023-07-04 00:00:00+00:00,True,9
4,b_76 4,76,0,US36 & Bfld,1241914,SU_merged_114569133,114458891,2023-05-28 00:00:00+00:00,2023-05-28 00:00:00+00:00,2023-08-19 00:00:00+00:00,...,20230819,6,sunday,1,6,NaN,NaN,NaT,True,8


In [67]:
data.feed_info.feed_version[0]

'Merged-31664M31661-20230415-035754-Jan23 and May23'

In [72]:
VERSION_ID = data.feed_info.feed_version[0]
def summarize_and_save(trip_summary): 
    # now group to get trips by hour by date by route
    route_daily_hourly_summary = trip_summary.groupby(by = ['raw_date', 'route_id', 'arrival_hour'])['trip_id'].count().reset_index()

    route_daily_hourly_summary.rename(columns = {'arrival_hour': 'hour', 'trip_id': 'trip_count', 'raw_date': 'date'}, inplace = True)
    route_daily_hourly_summary.date = route_daily_hourly_summary.date.dt.date
    if BUCKET_TYPE == "private":
        route_daily_hourly_summary.to_csv(f's3://rtd-ghost-buses-{BUCKET_TYPE}/schedule_summaries/route_level/schedule_route_daily_hourly_summary_{VERSION_ID}.csv', index = False)
    
    # now group to get trips by hour by date by route by *direction*
    route_dir_daily_hourly_summary = trip_summary.groupby(by = ['raw_date', 'route_id', 'direction_id', 'arrival_hour'])['trip_id'].count().reset_index()

    route_dir_daily_hourly_summary.rename(columns = {'arrival_hour': 'hour', 'trip_id': 'trip_count', 'raw_date': 'date'}, inplace = True)
    route_dir_daily_hourly_summary.date = route_dir_daily_hourly_summary.date.dt.date
    if BUCKET_TYPE == "private":
        route_dir_daily_hourly_summary.to_csv(f's3://rtd-ghost-buses-{BUCKET_TYPE}/schedule_summaries/route_dir_level/schedule_route_dir_daily_hourly_summary_{VERSION_ID}.csv', index = False)

In [73]:
summarize_and_save(trip_summary)

## Most common shape by route

In [78]:
# get trip count by route, direction, shape id
trips_by_rte_direction = data.trips.groupby(['route_id', 'shape_id', 'direction_id'])['trip_id'].count().reset_index()

In [79]:
# keep only most common shape id by route, direction
# follow: https://stackoverflow.com/a/54041328
most_common_shapes = trips_by_rte_direction.sort_values('trip_id').drop_duplicates(['route_id','direction_id'],keep='last')

In [80]:
# get additional route attributes
most_common_shapes = most_common_shapes.merge(data.routes, how = 'left', on = 'route_id')

In [81]:
# make shapely points
# https://www.geeksforgeeks.org/apply-function-to-every-row-in-a-pandas-dataframe/
data.shapes['pt'] = data.shapes.apply(lambda row: shapely.geometry.Point((float(row['shape_pt_lon']), float(row['shape_pt_lat']))), axis = 1)

c:\Users\Chris\.conda\envs\rtd_data_env\lib\site-packages\pandas\core\dtypes\cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [82]:
data.shapes['shape_pt_sequence'] = pd.to_numeric(data.shapes['shape_pt_sequence'])

In [83]:
# construct sorted list of shapely points
# custom aggregation function: https://stackoverflow.com/a/10964938

def make_linestring_of_points(sub_df):
    sorted_df = sub_df.sort_values(by = 'shape_pt_sequence')
    return shapely.geometry.LineString(list(sorted_df['pt']))

constructed_shapes = data.shapes.groupby('shape_id').apply(make_linestring_of_points).reset_index()

c:\Users\Chris\.conda\envs\rtd_data_env\lib\site-packages\pandas\core\dtypes\cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [84]:
# merge in the other route attributes
final = most_common_shapes.merge(constructed_shapes, how = 'left', on = 'shape_id')

In [85]:
# make a "geometry" column for geopandas
final['geometry'] = final[0]

In [86]:
# construct the geopandas geodataframe
final_gdf = geopandas.GeoDataFrame(data = final)

In [87]:
# drop the column that's a list of shapely points
final_gdf = final_gdf.drop(0, axis = 1)

In [88]:
# https://gis.stackexchange.com/questions/11910/meaning-of-simplifys-tolerance-parameter
final_gdf['geometry'] = final_gdf['geometry'].simplify(.0001)

In [89]:
# save to file as geojson (this saves locally)
with open('route_shapes_simplified_linestring.geojson', 'w') as f:
    f.write(final_gdf.loc[(final_gdf['route_type'] == '3')].to_json())

# Exploratory

In [37]:
data.stop_times.loc[data.stop_times['trip_id']=='114354115']
# data.calendar
# find trip_id's where stop_id = 34327 and stop_sequence = 1
data.stop_times.loc[(data.stop_times['stop_id']=='34327') & (data.stop_times['stop_sequence']=='1') ]
# 526!
# departure_time == "07:01:00" #?
data.stop_times.loc[(data.stop_times['stop_id']=='34327') & (data.stop_times['stop_sequence']=='1') & (data.stop_times['departure_time']=="07:01:00" ) ]

data.stop_times.loc[data.stop_times['trip_id']=='114450962']



,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
116263,114450962,07:01:00,07:01:00,34327,1,NaN,NaN,1,NaN,NaN
116264,114450962,07:05:55,07:05:55,20745,2,NaN,NaN,NaN,NaN,NaN
116265,114450962,07:07:00,07:07:00,17717,3,NaN,NaN,NaN,NaN,NaN
116266,114450962,07:09:00,07:09:00,17714,4,NaN,NaN,NaN,NaN,NaN
116267,114450962,07:10:46,07:10:46,17711,5,NaN,NaN,NaN,NaN,NaN
116268,114450962,07:13:02,07:13:02,17730,6,NaN,NaN,NaN,NaN,NaN
116269,114450962,07:15:00,07:15:00,12849,7,NaN,NaN,NaN,NaN,NaN
116270,114450962,07:15:44,07:15:44,12978,8,NaN,NaN,NaN,NaN,NaN
116271,114450962,07:17:10,07:17:10,12953,9,NaN,NaN,NaN,NaN,NaN
116272,114450962,07:17:52,07:17:52,12861,10,NaN,NaN,NaN,NaN,NaN
